# 使用聊天模型和提示模板构建一个简单的 LLM 应用程序

在本快速入门中，我们将向您展示如何使用 LangChain 构建一个简单的 LLM 应用程序。此应用程序会将文本从英语翻译成另一种语言。这是一个相对简单的 LLM 应用程序 - 它只是一个 LLM 调用加上一些提示。尽管如此，这仍然是开始使用 LangChain 的好方法 - 只需一些提示和 LLM！

## 安装Langchain

In [ ]:
! pip install langchain==0.3.9

## 加载环境变量

In [ ]:
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。
# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
env = load_dotenv(find_dotenv())

## LangSmith

您使用 LangChain 构建的许多应用程序将包含多个步骤，其中包含多次调用 LLM 调用。随着这些应用程序变得越来越复杂，能够检查您的链条或代理内部到底发生了什么变得至关重要。最好的方法是使用 LangSmith。

In [ ]:
#　这是langchain提供的一种调用的追踪平台，能看到调用链路的输入和输出
import os

print(os.environ["LANGCHAIN_TRACING_V2"]) 
print(os.environ["LANGCHAIN_API_KEY"])

## 创建语言模型

In [ ]:
!pip install -qU langchain-openai

In [ ]:
from langchain_openai import ChatOpenAI

base_url = os.environ["BASE_URL"]
api_key = os.environ["API_KEY"]
model = ChatOpenAI(model="gpt-4o-mini", api_key=api_key, base_url=base_url)

我们首先直接使用模型。ChatModel 是 LangChain Runnables 的实例，这意味着它们公开了用于与它们交互的标准接口。要简单地调用模型，我们可以将消息列表传递给 .invoke 方法。

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("将以下内容从中文翻译成英文"),
    HumanMessage("你好吗？"),
]

model.invoke(messages)

In [ ]:
# 流式输出
for token in model.stream(messages):
    print(token.content, end="|")

## 提示模板

现在，我们将消息列表直接传递到语言模型中。此消息列表来自何处？通常，它是由用户输入和应用程序逻辑的组合构建的。此应用程序逻辑通常采用原始用户输入，并将其转换为准备传递给语言模型的消息列表。常见转换包括添加系统消息或使用用户输入设置模板格式。

提示模板是 LangChain 中的一个概念，旨在协助进行这种转换。它们接收原始用户输入并返回准备传递到语言模型的数据（提示）。

language：要将文本翻译成的语言

text： 要翻译的文本

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "将以下内容从 中文 翻译成  {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [ ]:
prompt = prompt_template.invoke({"language":"英语","text":"你好吗？"})
prompt.to_messages()

In [ ]:
response = model.invoke(prompt)
print(response.content)